# Intermediate Examples

In [ ]:
import numpy as np
import libfmp.b as lfb
import matplotlib.pyplot as plt
from makeplotplayable import Session

from example_data import simple_annotations_file, simple_audio_file

sr = 48000
annotations = np.genfromtxt(simple_annotations_file, delimiter=',')
audio, _ = lfb.read_audio(simple_audio_file, mono=True, Fs=sr)

# Example 1
This example shows an alternative way to wrap the plotting function and how the original can be used too.
And the two alternative return types of the plotting function
In addition, the way how to stream audio directly from a file is shown

In [ ]:
# noinspection PyShadowingNames
def plot(annotations, disable_cursor=False):
    fig, ax = plt.subplots()
    ax.plot([0, annotations[-1,0]])
    ax.set_xlabel("number of files played")
    ax.set_ylabel("time in seconds")

    # external plot doesn't need to be interactive
    if disable_cursor:
        return

    # configuration parameters are optional
    return fig, ax


# create session from file
session01 = Session.from_file(simple_audio_file, looping=True)
#place curser at 10s
session01.time = 10

#start the session to enable audio playback
session01.start()

#call the plot function plot will be show normally
plot(annotations)

# wrap the plot function to create a playable version
plot_wrap = session01(plot)
# call that playable version
plot_wrap(annotations)
# call playable version with playback disabled
plot_wrap(annotations, disable_cursor=True)

In [ ]:
# It is possible to query the current playback time and state
print(f'Session state: {session01.time=} {session01.paused=}')

# Example 2
This example shows that multiple plotting functions are possible.
How to use extra functions.
The way the "sparse" mapping works (mapping as a list of time position pairs)
and additional plot parameters

In [ ]:
# create session from file
session02 = Session(audio, sr)
session02.start()

# noinspection PyShadowingNames
@session02
def plot_a(audio, sr):
    import libfmp.b as lfb

    fig, ax, line = lfb.plot_signal(audio, sr)

    return fig, ax


# noinspection PyShadowingNames
@session02
def plot_b(audio, sr):
    fig, ax = plt.subplots()

    duration = audio.shape[0] / sr
    mapping = mapper(duration)

    ax.plot(mapping[:,1],mapping[:,0])
    ax.update_datalim([(0, duration), (duration, 0)])
    ax.set_xlabel("some position axis")
    ax.set_ylabel("time in seconds")

    return fig, ax, {"mapping": mapping,
                     "axvline_kwargs": {"alpha": 0.9, "ls": 'dashdot', "color": 'c', "lw": 2, "zorder": 10},
                     "title": "B02 Sparse Mapping Function",
                     "window_pos": (0, 0)}


def mapper(duration):
    # mapping the midpoint offset
    # after half the audio is played the curser is still at the 10% mark
    mapping = np.array([
        [0, 0],
        [duration / 2, duration / 10],
        [duration, duration]
    ])
    return mapping


# it is possible to use different plotting functions
plot_a(audio, sr)
plot_b(audio, sr)

# Example 3
This example shows how an extra axes for a cursor could be constructed
It doesn't show any new core features

In [ ]:
session03 = Session(audio, sr)
session03.start()

# noinspection PyShadowingNames
@session03
def plot(audio, sr, annotations):
    from matplotlib.transforms import Bbox

    fig, axs = plt.subplots(2)

    # first subplot
    lfb.plot_signal(audio, sr, ax=axs[0])
    # second subplot
    axs[1].plot(annotations)
    axs[1].set_xlim(annotations[0, 0], annotations[-1, 0])
    axs[1].set_ylabel("some data")

    fig.tight_layout()

    # create bounding box for extra axes
    pa = axs[0].get_position().get_points()
    pb = axs[1].get_position().get_points()
    p = Bbox([[pb[0, 0], pb[0, 1]], [pa[1, 0], pa[1, 1]]])

    # create axes (spanning both plots, with an arbitrary non-zero x-axis)
    ax = fig.add_axes(p, xlim=(0, 1))
    ax.update_datalim([(0, 0), (1, 0)])
    # hide decoration
    ax.axis('off')

    return fig, ax, {"title": "B03 Subplot Cursor"}


plot(audio, sr, annotations)

# Example 4:
This example shows how extra elements can be animated and how to register mpl events

In [ ]:
session04 = Session(audio, sr)
session04.start()

# noinspection PyShadowingNames
@session04
def plot(x, sr):
    fig, ax, line = lfb.plot_signal(x, sr)

    # this element will be animated (is instance of Artist)
    text = ax.text(10, 0, '____', style='italic', bbox={
        'facecolor': 'red', 'alpha': 0.5, 'pad': 10})

    string = ""
    force_redraw = False

    # a simple demo of the mpl key_press_event; write into a text box
    def on_key(event):
        # nonlocal allows to override variables from the outer functions scope similar to global
        nonlocal string, force_redraw
        force_redraw = True
        if event.key == "delete":
            string = ""
        elif event.key == "backspace":
            if len(string) > 0:
                string = string[:-1]
        else:
            string += event.key

    fig.canvas.mpl_connect('key_press_event', on_key)

    last_draw_input = None

    # called every (potential) frame (returns if redraw is needed)
    def draw_function(time, pos, paused):
        nonlocal last_draw_input, force_redraw
        # only redraw if something animated changed
        if last_draw_input == (time, pos, paused) and not force_redraw:
            return False

        last_draw_input = (time, pos, paused)

        text.set_text(f"{time=:3.2f}, {pos=:3.2f}, {paused=} ♯ \N{Music Flat Sign} {string}")

        force_redraw = False
        return True

    return fig, ax, {"title": "B04 Extra Animations",
                     "draw_function": draw_function,
                     "artists": [text]}

plot(audio, sr)